## import

In [252]:
import numpy as np
import sys,os
import random
import matplotlib.pyplot as plt
import csv
import ast
from collections import OrderedDict

## Input Data 생성

x, y, z 데이터 중 

x가 0~99이면    정답 => [1,0,0]

x가 100~199이면 정답 => [0,1,0]

x가 200~299이면 정답 => [0,0,1]

In [232]:
input_data=np.empty((0,3))
t=np.empty((0,3))
for i in range(1000):
    input_data=np.append(input_data, np.array([[random.randrange(0,300) for j in range(3)]]),axis=0)
    x=input_data[-1,0]
    if x<100:
        t=np.append(t,np.array([[1, 0, 0]]),axis=0)
    elif x<200:
        t=np.append(t,np.array([[0, 1, 0]]),axis=0)
    elif x<300:
        t=np.append(t,np.array([[0, 0, 1]]),axis=0)
print(input_data.shape)

(1000, 3)


## Input Data Normalize  (0~1)

정규화 식


<img src = "../simple_network/Data/normalize.png" width = "200" height = "75">



In [233]:
input_max=299
input_min=0
de=input_max-input_min
normalize_input=np.zeros((1000,3),dtype=float)
for i in range(1000):
    for j in range(3):
        normalize_input[i,j]=(input_data[i,j]-input_min)/de
print(input_data)
print(normalize_input)

[[119. 120. 239.]
 [ 94.   8. 173.]
 [239. 112. 182.]
 ...
 [262. 177. 131.]
 [268.  55. 188.]
 [148.  12. 155.]]
[[0.39799331 0.40133779 0.7993311 ]
 [0.31438127 0.02675585 0.57859532]
 [0.7993311  0.37458194 0.60869565]
 ...
 [0.87625418 0.59197324 0.43812709]
 [0.89632107 0.18394649 0.62876254]
 [0.49498328 0.04013378 0.51839465]]


### Input Data 파일에 저장

In [234]:
np.savez("../simple_network/Data/input_data_set.npz", input_data)
np.savez("../simple_network/Data/t_set.npz", t)

#### Input Data 파일 읽기 테스트

np.savez()를 사용하여 numpy array 형 변수 읽어오기


In [235]:
read_input=np.load("../simple_network/Data/input_data_set.npz", allow_pickle=True)
read_t=np.load("../simple_network/Data/t_set.npz", allow_pickle=True)
r_input_data=read_input["arr_0"]
r_t_data=read_t["arr_0"]
print(r_input_data)
print(r_t_data)

[[119. 120. 239.]
 [ 94.   8. 173.]
 [239. 112. 182.]
 ...
 [262. 177. 131.]
 [268.  55. 188.]
 [148.  12. 155.]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


## 가중치 W, 편향 B  생성 ##
1. 정규분포를 따르는 랜덤수로 생성   (np.random.randn()을 사용하면 "평균 = 0, 표준편차 = 1"인 랜덤수가 생성됨.)
    
    0.01을 곱해주는 이유: w의 초기값이크면 gradient vanishing 문제가 발생할 수 있음.

    따라서 초기 가중치 값에 0.01을 곱해 표준편차를 0.01로 만들어줌.


In [264]:
weight_init_std=0.01
params={}
params['W1']=weight_init_std*np.random.randn(3,4)
params['B1']=np.zeros(4)
params['W2']=weight_init_std*np.random.randn(4,4)
params['B2']=np.zeros(4)
params['W3']=weight_init_std*np.random.randn(4,3)
params['B3']=np.zeros(3)


{'W1': array([[ 0.1594863 , -1.71468747,  0.21852038,  1.31476412],
       [ 0.28219757, -0.84008366, -0.97997099,  0.27220219],
       [ 1.14944062, -1.40832679,  0.09631079, -0.04712944]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[-0.00895927,  0.01202074, -0.00244403,  0.00181983],
       [-0.01338409, -0.00516968, -0.00660834,  0.00305399],
       [-0.01529869, -0.00705336, -0.02191124,  0.00092591],
       [-0.01276128,  0.00634618,  0.00833959, -0.0064241 ]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[-0.0010002 ,  0.00248384,  0.01324728],
       [ 0.00707038, -0.00249271, -0.01158886],
       [ 0.00051597, -0.0027449 ,  0.01371886],
       [-0.00296105, -0.00218017,  0.00459657]]), 'B3': array([0., 0., 0.])}


2. constant로 생성

In [272]:
params['W1']=np.zeros((3,4))
params['B1']=np.zeros(4)
params['W2']=np.zeros((4,4))
params['B2']=np.zeros(4)
params['W3']=np.zeros((4,3))
params['B3']=np.zeros(3)

params['W1'].fill(0.1)
params['W2'].fill(0.1)
params['W3'].fill(0.1)


3. 균일분포를 따르는 랜덤수로 생성

In [274]:
weight_init_std=0.01
params={}
params['W1']=np.random.uniform(-1,1,(3,4))
params['B1']=np.zeros(4)
params['W2']=np.random.uniform(-1,1,(4,4))
params['B2']=np.zeros(4)
params['W3']=np.random.uniform(-1,1,(4,3))
params['B3']=np.zeros(3)
print(params)

{'W1': array([[-0.89253291, -0.20755451, -0.66512836,  0.64380782],
       [ 0.40105725,  0.76615519,  0.93315021,  0.54949523],
       [ 0.98846617,  0.22953977, -0.92574079, -0.97149697]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[-0.31579225,  0.64694344,  0.73226941,  0.92162506],
       [-0.86975706, -0.91085778,  0.82656719, -0.3899066 ],
       [ 0.1159748 ,  0.96488977, -0.19910293,  0.3317428 ],
       [-0.19824087,  0.53638933,  0.05542945, -0.52495372]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[-0.4573878 , -0.48388157,  0.06464066],
       [ 0.40637803,  0.8985598 ,  0.38817475],
       [ 0.56238569, -0.66214777, -0.25187475],
       [-0.17243956,  0.37276046, -0.40821605]]), 'B3': array([0., 0., 0.])}


## 가중치 W, 편향 B weight_bias.npz 파일에 저장

np.savez() --> 여러개의 배열을 1개의 압축되지 않은 *.npz 포맷 파일로 저장하기 (Save several arrays into a single file in uncompressed .npz format)

In [275]:
np.savez("../simple_network/Data/weight_bias.npz", **params)

##### dictionary형 변수 csv파일에 저장

In [239]:
# open a file in write mode
with open('../simple_network/Data/weight_bias.csv', 'w', newline='') as f:
    # create a csv writer
    writer = csv.writer(f)
    # write the keys of the dictionary as the header row
    writer.writerow(params.keys())
    # write the values of the dictionary as a single row
    writer.writerow(params.values())


npz 변수 파일 불러오기

np.load("파일명",allow_pickle=True)

In [240]:
my_dictionary = np.load("../simple_network/Data/weight_bias.npz", allow_pickle=True)
paaa=dict(my_dictionary)
print(paaa)
print(params)

{'W1': array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]]), 'B3': array([0., 0., 0.])}
{'W1': array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 'B1': array([0., 0., 0., 0.]), 'W2': array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 'B2': array([0., 0., 0., 0.]), 'W3': array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]]), 'B3': array([0., 0., 0.])}
